In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Stop warnings
# -------------
import warnings
warnings.filterwarnings("ignore")

# General imports
# ---------------
import os
import sys
import json
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.colors as colors
import ipdb
import platform
opj = os.path.join
deb = ipdb.set_trace
%matplotlib inline  

In [2]:
# MRI imports
# -----------
import nibabel as nb
import cortex

subject = '999999'

# Define analysis parameters
# --------------------------
with open('../../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)

# Define cluster/server specific parameters
# -----------------------------------------
if 'aeneas' in platform.uname()[1]:
    base_dir = analysis_info['aeneas_base_folder']
    main_cmd = '/home/szinte/software/workbench/bin_rh_linux64/wb_command'
elif 'local' in platform.uname()[1]:
    base_dir = analysis_info['local_base_folder']
    main_cmd = '/Applications/workbench/bin_macosx64/wb_command'

fit_model = 'gauss'
deriv_dir = opj(base_dir,'pp_data',subject,fit_model,'deriv')

In [3]:
# Creating zoom picture
# ---------------------
def zoom_to_roi(subject, roi, hem, margin=1.0) :
    roi_verts = cortex.get_roi_verts(subject, roi)[roi]
    roi_map = cortex.Vertex.empty(subject)
    roi_map.data[roi_verts] = 1

    (lflatpts, lpolys), (rflatpts, rpolys) = cortex.db.get_surf(subject, "flat",
                                                                nudge=True)
    sel_pts = dict(left=lflatpts, right=rflatpts)[hem]
    roi_pts = sel_pts[np.nonzero(getattr(roi_map, hem))[0],:2]
    
    xmin, ymin = roi_pts.min(0) - margin
    xmax, ymax = roi_pts.max(0) + margin
    zoom_width = roi_pts.max(0)[0]- roi_pts.min(0)[0]
    zoom_height = roi_pts.max(0)[1]- roi_pts.min(0)[1]
    sqr_zoom_size = np.round(np.max([zoom_width,zoom_height]))*1.1
    zoom_ctr_x = np.mean([roi_pts.max(0)[0],roi_pts.min(0)[0]])
    zoom_ctr_y = np.mean([roi_pts.max(0)[1],roi_pts.min(0)[1]])
    mat = [zoom_ctr_x-sqr_zoom_size/2.0,zoom_ctr_x+sqr_zoom_size/2.0,zoom_ctr_y-sqr_zoom_size/2.0,zoom_ctr_y+sqr_zoom_size/2.0]
    zoom_plot = plt.axis(mat)

def draw_cortex_vertex(subject,data,cmap,vmin,vmax,cbar = 'discrete',cmap_steps = 255,\
                        alpha = None,depth = 1,thick = 1,height = 1024,sampler = 'nearest',\
                        with_curvature = True,with_labels = True,with_colorbar = False,\
                        with_borders = False,curv_brightness = 0.95,curv_contrast = 0.05,add_roi = False,\
                        roi_name = 'empty',col_offset = 0):
    import cortex
    import pdb
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.colors as colors
    from matplotlib import cm
    import matplotlib as mpl
    import ipdb
    deb = ipdb.set_trace

    # define colormap
    base = cortex.utils.get_cmap(cmap)
    val = np.fmod(np.linspace(0+col_offset, 1+col_offset,cmap_steps+1,endpoint=False),1.0)
    colmap = colors.LinearSegmentedColormap.from_list(  'my_colmap',
                                                        base(val),
                                                        N = cmap_steps)

    # convert data to RGB
    vrange = float(vmax) - float(vmin)
    norm_data = ((data-float(vmin))/vrange)*cmap_steps
    mat = colmap(norm_data.astype(int))

    vertex_rgb = cortex.VertexRGB(  red = mat[...,0],
                                    green = mat[...,1],
                                    blue = mat[...,2],
                                    alpha = alpha,
                                    subject = subject,
                                    )
    
    return vertex_rgb

In [4]:
# Create derivatives flatmaps
# ---------------------------
cmap_polar = 'hsv'
col_offset = 1/14.0
sign_idx, rsq_idx, ecc_idx, polar_real_idx, polar_imag_idx , size_idx, \
            non_lin_idx, amp_idx, baseline_idx, cov_idx, x_idx, y_idx = 0,1,2,3,4,5,6,7,8,9,10,11

mask_dir  = 'neg'
# Combine hemispheres
deriv_mat=[]
for hemi in ['L','R']:
    deriv_file = nb.load(opj(deriv_dir,mask_dir,"prf_deriv_{hemi}_{mask_dir}_fsaverage.func.gii".format(hemi = hemi, mask_dir = mask_dir)))
    deriv_mat.append(np.array([deriv_file.darrays[i].data for i in range(len(deriv_file.darrays))]))
deriv_mat = np.hstack(deriv_mat)
deriv_mat = deriv_mat.T
threshold_mask = np.logical_and(np.logical_and( deriv_mat[:,rsq_idx]>=analysis_info['rsq_threshold'],
                                                deriv_mat[:,cov_idx]>=analysis_info['cov_threshold']),
                                                deriv_mat[:,size_idx]>=analysis_info['size_threshold'])

# R-square
rsq_data = deriv_mat[:,rsq_idx]
rsq_data[~threshold_mask] = np.nan
alpha = rsq_data

# Polar angle
real_num = deriv_mat[:,polar_real_idx]
real_num[~threshold_mask] = np.nan
imag_num = deriv_mat[:,polar_imag_idx]
imag_num[~threshold_mask] = np.nan
pol_comp_num = real_num + 1j * imag_num
polar_ang = np.angle(pol_comp_num)
ang_norm = (polar_ang + np.pi) / (np.pi * 2.0)

cmap_steps = 255
param_polar = {'data': ang_norm.T, 'cmap': cmap_polar, 'alpha': alpha.T, 'vmin': 0, 'vmax': 1, 'cmap_steps': cmap_steps,\
                       'curv_brightness': 0.05, 'curv_contrast': 0.1, 'cbar': 'polar', 'col_offset': col_offset,\
                        'subject': 'fsaverage','add_roi': False}
vertex_polar = draw_cortex_vertex(**param_polar)

# plot_in = cortex.quickshow(vertex_polar, depth = 1,thick = 0,height = 768,sampler = 'nearest',\
#                         with_curvature = True,with_labels = False,with_colorbar = False,\
#                         with_borders = False,curv_brightness = 0.95,curv_contrast = 0.05)
# zoom_to_roi('fsaverage', 'ANG', 'left')
# plt.close(plot_in)
# plot_in.savefig(opj(base_dir,'pp_data',subject,fit_model,'figs','roi','neg','ANG_left.png'),bbox_inches = 'tight')

# plot_in = cortex.quickshow(vertex_polar, depth = 1,thick = 0,height = 768,sampler = 'nearest',\
#                         with_curvature = True,with_labels = False,with_colorbar = False,\
#                         with_borders = False,curv_brightness = 0.95,curv_contrast = 0.05)
# zoom_to_roi('fsaverage', 'ANG', 'right')
# plt.close(plot_in)
# plot_in.savefig(opj(base_dir,'pp_data',subject,fit_model,'figs','roi','neg','ANG_right.png'),bbox_inches = 'tight')

In [5]:
# Bokeh import
# ------------
from bokeh.plotting import figure 
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.models.annotations import Span, Label
from bokeh.layouts import row, column, gridplot
from bokeh.models import BoxZoomTool, BoxSelectTool, Spacer, WheelZoomTool, PanTool, ResetTool
from bokeh.models.glyphs import Text
from bokeh.models.mappers import LinearColorMapper
from bokeh.io import output_notebook, show,save, output_file, export_png, export_svgs
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
import matplotlib.image as mpimg
import wquantiles
output_notebook()

# Figure settings
rot_left = -10
rot_right = -10

# Zoom ROI in bokeh
# -----------------
# load image
left_filename = opj(base_dir,'pp_data',subject,fit_model,'figs','roi','neg','ANG_left.png')
img_left = mpimg.imread(left_filename)
img_left = img_left*255.0
img_left = np.flipud(img_left)

right_filename = opj(base_dir,'pp_data',subject,fit_model,'figs','roi','neg','ANG_right.png')
img_right = mpimg.imread(right_filename)
img_right = img_right*255.0
img_right = np.flipud(img_right)

def roi_coord_mask(roi,hem):
    # get data
    roi_verts = cortex.get_roi_verts('fsaverage', roi)[roi]
    roi_map = cortex.Vertex.empty('fsaverage')
    roi_map.data[roi_verts] = 1
    (lflatpts, lpolys), (rflatpts, rpolys) = cortex.db.get_surf('fsaverage', "flat",nudge=True)
    sel_pts = dict(left=lflatpts, right=rflatpts)[hem]
    roi_pts = sel_pts[np.nonzero(getattr(roi_map, hem))[0],:2]
    if hem == 'left':
        roi_mask = np.hstack([roi_map.left[:],roi_map.right[:]*0])
    elif hem == 'right':
        roi_mask = np.hstack([roi_map.left[:]*0,roi_map.right[:]])
    roi_mask = roi_mask==1
    
    return(roi_pts,roi_mask)

def get_colors(data,cmap,cmap_steps,col_offset,vmin,vmax):
    base = cortex.utils.get_cmap(cmap)
    val = np.fmod(np.linspace(0 + col_offset, 1 + col_offset, cmap_steps + 1, endpoint = False), 1.0)
    colmap = colors.LinearSegmentedColormap.from_list('my_colmap', base(val), N = cmap_steps)
    vrange = float(vmax) - float(vmin)
    norm_data = (( data - float(vmin) ) / vrange) * cmap_steps 
    col_mat_rgb = colmap(norm_data.astype(int)) * 255.0
    colors_val_rgb = ["#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b in zip(col_mat_rgb[:,0], col_mat_rgb[:,1], col_mat_rgb[:,2])]
    return colors_val_rgb

def rotate_pts(pts,orig,rot_deg):
    rot = np.radians(rot_deg)
    qx = orig[0] + np.cos(rot) * (pts[0] - orig[0]) + np.sin(rot) * (pts[1] - orig[1])
    qy = orig[1] + -np.sin(rot) * (pts[0] - orig[0]) + np.cos(rot) * (pts[1] - orig[1])
    rot_pts = [qx,qy]
    return rot_pts

arrow_start = [0.5,0.2]
arrow_end = [0.5,0.8]
arrow_ctr = [0.5,0.5]
p_width = 400
p_height = 400
min_border_large = 10
stim_color = tuple([250,250,250])
bg_color = tuple([229,229,229])
valtick = (np.linspace(0,1,11))

x_range_zoom = (0,1)
y_range_zoom = (0,1)
x_label_zoom = 'ROI short axis (norm.)'
y_label_zoom = 'ROI long axis (norm.)'

# Left ANG
# --------
zoom_left = figure( plot_width = p_width,
                    plot_height = p_height,
                    x_range = x_range_zoom,
                    y_range = y_range_zoom,
                    min_border_left = min_border_large,
                    min_border_right = min_border_large,
                    min_border_bottom = min_border_large,
                    min_border_top = min_border_large,
                    toolbar_location = None,
                    title = '')

zoom_left.xaxis.axis_label = x_label_zoom
zoom_left.yaxis.axis_label = y_label_zoom
zoom_left.grid.grid_line_color = None
zoom_left.axis.minor_tick_in = False
zoom_left.axis.minor_tick_out = False
zoom_left.axis.major_tick_in = False
zoom_left.outline_line_alpha = 0
xaxis_ticker = np.linspace(0.06,0.93,11)
zoom_left.xaxis.ticker = xaxis_ticker

dict_label = {}
for val_num, val in enumerate(xaxis_ticker):
    dict_label.update({val: '%1.0g'%valtick[val_num]})
zoom_left.xaxis.ticker = xaxis_ticker    
zoom_left.xaxis.major_label_overrides = dict_label
yaxis_ticker = np.linspace(0.032,0.97,11)
valtick = (np.linspace(0,1,11))
dict_label = {}
for val_num, val in enumerate(yaxis_ticker):
    dict_label.update({val: '%1.0g'%valtick[val_num]})
zoom_left.yaxis.ticker = yaxis_ticker    
zoom_left.yaxis.major_label_overrides = dict_label
zoom_left.background_fill_color = bg_color
zoom_left.axis.axis_label_standoff = 10
zoom_left.axis.axis_label_text_font_style = 'normal'

# draw roi
zoom_left.image_rgba(image=[img_left.astype('uint8')], x=-0.075, y=-0.06, dw=1.1,dh=1.1)

# draw arrow
arrow_start_left = rotate_pts(arrow_start,arrow_ctr,rot_left)
arrow_end_left = rotate_pts(arrow_end,arrow_ctr,rot_left)
zoom_left.add_layout(Arrow(end = NormalHead(size = 10,fill_color = 'black', line_color = 'black',line_width = 6), 
                            x_start = arrow_start_left[0], y_start = arrow_start_left[1],
                            x_end = arrow_end_left[0], y_end = arrow_end_left[1], line_color = 'black',line_width = 6))
zoom_left.add_layout(Arrow(end = NormalHead(size = 10,fill_color = 'white', line_color= 'white',line_width = 2), 
                            x_start = arrow_start_left[0], y_start = arrow_start_left[1],
                            x_end = arrow_end_left[0], y_end = arrow_end_left[1],line_color = 'white',line_width = 2))

# draw title
zoom_left.text(x = 0.05, y = 0.95, text = ['ANG LH'],text_color = 'white',text_font_size = '10pt', text_align = 'left', text_baseline = 'middle')

# Right ANG
# ---------
zoom_right = figure(plot_width = p_width,
                    plot_height = p_height,
                    x_range = x_range_zoom,
                    y_range = y_range_zoom,
                    min_border_left = min_border_large,
                    min_border_right = min_border_large,
                    min_border_bottom = min_border_large,
                    min_border_top = min_border_large,
                    toolbar_location = None,
                    title = '')

zoom_right.xaxis.axis_label = x_label_zoom
zoom_right.yaxis.axis_label = y_label_zoom
zoom_right.grid.grid_line_color = None
zoom_right.axis.minor_tick_in = False
zoom_right.axis.minor_tick_out = False
zoom_right.axis.major_tick_in = False
zoom_right.outline_line_alpha = 0

xaxis_ticker = np.linspace(0.17,0.83,11)
zoom_right.xaxis.ticker = xaxis_ticker
dict_label = {}
for val_num, val in enumerate(xaxis_ticker):
    dict_label.update({val: '%1.0g'%valtick[val_num]})
zoom_right.xaxis.ticker = xaxis_ticker    
zoom_right.xaxis.major_label_overrides = dict_label
yaxis_ticker = np.linspace(0.035,0.975,11)
valtick = (np.linspace(0,1,11))
dict_label = {}
for val_num, val in enumerate(yaxis_ticker):
    dict_label.update({val: '%1.0g'%valtick[val_num]})
zoom_right.yaxis.ticker = yaxis_ticker    
zoom_right.yaxis.major_label_overrides = dict_label
zoom_right.background_fill_color = bg_color
zoom_right.axis.axis_label_standoff = 10
zoom_right.axis.axis_label_text_font_style = 'normal'

# draw roi
zoom_right.image_rgba(image=[img_right.astype('uint8')], x=-0.075, y=-0.06, dw=1.1,dh=1.1)


Loading BokehJS ...

GlyphRenderer(id='4dd55793-a0d9-45a9-81d3-18a803649ea7', ...)

In [6]:

# draw arrow
arrow_start_right = rotate_pts(arrow_start,arrow_ctr,rot_right)
arrow_end_right = rotate_pts(arrow_end,arrow_ctr,rot_right)
zoom_right.add_layout(Arrow(end = NormalHead(size = 10,fill_color = 'black', line_color= 'black',line_width=6), 
                            x_start = arrow_start_right[0], y_start = arrow_start_right[1],
                            x_end = arrow_end_right[0], y_end = arrow_end_right[1],line_color = 'black',line_width=6))
zoom_right.add_layout(Arrow(end = NormalHead(size = 10,fill_color = 'white', line_color= 'white',line_width=2), 
                            x_start = arrow_start_right[0], y_start = arrow_start_right[1],
                            x_end = arrow_end_right[0], y_end = arrow_end_right[1],line_color = 'white',line_width=2))

# draw title
zoom_right.text(x = 0.05, y = 0.95, text = ['ANG RH'],text_color = 'white',text_font_size = '10pt', text_align = 'left', text_baseline = 'middle')

# ROI rotation
# ------------
def rot_coord(coord,rot_deg):
    theta = np.radians(rot_deg)
    c, s = np.cos(theta), np.sin(theta)
    R = np.matrix([[c, -s], [s, c]])
    coord_rot = np.array(np.dot(coord,R))
    return coord_rot

# define left coordinates
ang_coord_left, ang_mask_left = roi_coord_mask('ANG','left')
alpha_left = alpha[ang_mask_left]
ang_norm_left = ang_norm[ang_mask_left]
real_num_left = real_num[ang_mask_left]
imag_num_left = imag_num[ang_mask_left]
ang_norm_left = ang_norm_left[~np.isnan(alpha_left)]
ang_norm_col_left = get_colors(ang_norm_left, 'hsv', 255, 1/14.0, 0, 1)
real_num_left = real_num_left[~np.isnan(alpha_left)]
imag_num_left = imag_num_left[~np.isnan(alpha_left)]
ang_coord_left = ang_coord_left[~np.isnan(alpha_left)]
ang_coord_rot_left = rot_coord(ang_coord_left,rot_left)
alpha_left = alpha_left[~np.isnan(alpha_left)]

# define right coordinates
ang_coord_right, ang_mask_right = roi_coord_mask('ANG','right')
alpha_right = alpha[ang_mask_right]
ang_norm_right = ang_norm[ang_mask_right]
real_num_right = real_num[ang_mask_right]
imag_num_right = imag_num[ang_mask_right]
ang_norm_right = ang_norm_right[~np.isnan(alpha_right)]
ang_norm_col_right = get_colors(ang_norm_right, 'hsv', 255, 1/14.0, 0, 1)
real_num_right = real_num_right[~np.isnan(alpha_right)]
imag_num_right = imag_num_right[~np.isnan(alpha_right)]
ang_coord_right = ang_coord_right[~np.isnan(alpha_right)]
ang_coord_rot_right = rot_coord(ang_coord_right,rot_right)
alpha_right = alpha_right[~np.isnan(alpha_right)]

# Polar progression computing
# -------------------------
nbins = 10
small_size_dot = 5
big_size_dot = small_size_dot*3

# ANG Left
# --------

# rename values
roi_long_left = ang_coord_rot_left[:,1]
weights_left = alpha_left
real_left = real_num_left
imag_left = imag_num_left
comp_num_left = real_left+ 1j*imag_left

# define value for scatter plot
scatter_roi_long_left_norm = (roi_long_left - np.min(roi_long_left))/(np.max(roi_long_left)-np.min(roi_long_left))
scatter_angle_left = np.angle(real_left + 1j * imag_left)
scatter_angle_norm_left = (scatter_angle_left + np.pi) / (np.pi * 2.0)
scatter_angle_norm_left_col = get_colors(scatter_angle_norm_left, 'hsv', 255, 1/14.0, 0, 1)
scatter_rsq_left_norm = (weights_left - np.min(weights_left))/(np.max(weights_left)-np.min(weights_left))
scatter_size_dots_left = np.round(small_size_dot+scatter_rsq_left_norm*big_size_dot)

# define binned values
bin_left = np.zeros(nbins+1)
roi_long_axis = np.linspace(0,1,nbins+1)
for tbin,bin_val in enumerate(roi_long_axis):
    bin_left[tbin] = wquantiles.quantile(data = roi_long_left, weights = weights_left, quantile = bin_val)

bin_rsq_left, bin_roi_long_left, bin_angle_left, bin_angle_norm_left = np.zeros(nbins), np.zeros(nbins), np.zeros(nbins), np.zeros(nbins)
bin_angle_left_ebx, bin_angle_left_std, bin_angle_left_95ci = [],[],[]
for bin_left_num in np.arange(0,nbins,1):
    val2pick = np.logical_and(roi_long_left >= bin_left[bin_left_num],roi_long_left <= bin_left[bin_left_num+1])
    # average
    bin_comp_num_left = np.average(a = comp_num_left[val2pick],weights = weights_left[val2pick])
    bin_angle_left[bin_left_num] = np.angle(bin_comp_num_left)
    bin_angle_norm_left[bin_left_num] = (bin_angle_left[bin_left_num] + np.pi) / (np.pi * 2.0)
    bin_rsq_left[bin_left_num] = np.average(a = weights_left[val2pick],weights = weights_left[val2pick])
    bin_roi_long_left[bin_left_num] = np.average(a = roi_long_left[val2pick], weights = weights_left[val2pick])
    
    # error bar
    bin_angle_left_ebx.append([ bin_roi_long_left[bin_left_num], bin_roi_long_left[bin_left_num]])
    bin_angle_left_std.append([ bin_angle_left[bin_left_num]-np.nanstd(np.angle(comp_num_left[val2pick])),\
                                bin_angle_left[bin_left_num]+np.nanstd(np.angle(comp_num_left[val2pick]))])

# normalization 
bin_angle_norm_left_col = get_colors(bin_angle_norm_left, 'hsv', 255, 1/14.0, 0, 1)
bin_rsq_left_norm = (bin_rsq_left - np.min(bin_rsq_left))/(np.max(bin_rsq_left)-np.min(bin_rsq_left))
bin_size_dots_left = np.round(small_size_dot+bin_rsq_left_norm*big_size_dot)
bin_roi_long_left_norm = (bin_roi_long_left - np.min(roi_long_left))/(np.max(roi_long_left)-np.min(roi_long_left))
bin_angle_left_ebx_norm = np.ndarray.tolist((bin_angle_left_ebx - np.min(roi_long_left))/(np.max(roi_long_left)-np.min(roi_long_left)))

# ANG Right
# ---------

# rename values
roi_long_right = ang_coord_rot_right[:,1]
weights_right = alpha_right
real_right = real_num_right
imag_right = imag_num_right

# add rotation to have it centered on 0 for plot
comp_num_right = real_right + 1j*imag_right
comp_num_rot_right = comp_num_right * (np.cos(np.pi)+1j*np.sin(np.pi))
real_right = np.real(comp_num_rot_right)
imag_right = np.imag(comp_num_rot_right)

comp_num_right = real_right+ 1j*imag_right

# define value for scatter plot
scatter_roi_long_right_norm = (roi_long_right - np.min(roi_long_right))/(np.max(roi_long_right)-np.min(roi_long_right))
scatter_angle_right = np.angle(real_right + 1j * imag_right)
scatter_angle_norm_right = (scatter_angle_right + np.pi) / (np.pi * 2.0)
scatter_angle_norm_right_col = get_colors(scatter_angle_norm_right, 'hsv', 255, 8/14.0, 0, 1)
scatter_rsq_right_norm = (weights_right - np.min(weights_right))/(np.max(weights_right)-np.min(weights_right))
scatter_size_dots_right = np.round(small_size_dot+scatter_rsq_right_norm*big_size_dot)

# define binned values
bin_right = np.zeros(nbins+1)
roi_long_axis = np.linspace(0,1,nbins+1)
for tbin,bin_val in enumerate(roi_long_axis):
    bin_right[tbin] = wquantiles.quantile(data = roi_long_right, weights = weights_right, quantile = bin_val)

bin_rsq_right, bin_roi_long_right, bin_angle_right, bin_angle_norm_right = np.zeros(nbins), np.zeros(nbins), np.zeros(nbins), np.zeros(nbins)
bin_angle_right_ebx, bin_angle_right_std, bin_angle_right_95ci = [],[],[]
for bin_right_num in np.arange(0,nbins,1):
    val2pick = np.logical_and(roi_long_right >= bin_right[bin_right_num],roi_long_right <= bin_right[bin_right_num+1])
    # average
    bin_comp_num_right = np.average(a = comp_num_right[val2pick],weights = weights_right[val2pick])
    bin_angle_right[bin_right_num] = np.angle(bin_comp_num_right)
    bin_angle_norm_right[bin_right_num] = (bin_angle_right[bin_right_num] + np.pi) / (np.pi * 2.0)
    bin_rsq_right[bin_right_num] = np.average(a = weights_right[val2pick],weights = weights_right[val2pick])
    bin_roi_long_right[bin_right_num] = np.average(a = roi_long_right[val2pick], weights = weights_right[val2pick])
    
    # error bar
    bin_angle_right_ebx.append([ bin_roi_long_right[bin_right_num], bin_roi_long_right[bin_right_num]])
    bin_angle_right_std.append([ bin_angle_right[bin_right_num]-np.nanstd(np.angle(comp_num_right[val2pick])),\
                                bin_angle_right[bin_right_num]+np.nanstd(np.angle(comp_num_right[val2pick]))])

# normalization 
bin_angle_norm_right_col = get_colors(bin_angle_norm_right, 'hsv', 255, 8/14.0, 0, 1)
bin_rsq_right_norm = (bin_rsq_right - np.min(bin_rsq_right))/(np.max(bin_rsq_right)-np.min(bin_rsq_right))
bin_size_dots_right = np.round(small_size_dot+bin_rsq_right_norm*big_size_dot)
bin_roi_long_right_norm = (bin_roi_long_right - np.min(roi_long_right))/(np.max(roi_long_right)-np.min(roi_long_right))
bin_angle_right_ebx_norm = np.ndarray.tolist((bin_angle_right_ebx - np.min(roi_long_right))/(np.max(roi_long_right)-np.min(roi_long_right)))


In [9]:
# Polar progression drawing
# -------------------------
x_range_pp = (0,1)
x_axis_ticker = np.linspace(x_range_pp[0],x_range_pp[1],11)
x_label_pp = 'Main direction (norm.)'

y_range_pp = (-np.pi, np.pi)
y_axis_ticker = np.linspace(y_range_pp[0],y_range_pp[1],5)
y_label_pp = 'Polar angle relative to opposite hemifield (rad)'

bin_angle_left_eb = bin_angle_left_std
bin_angle_right_eb = bin_angle_right_std

# Left ANG polar progression
# --------------------------
pp_left = figure(   plot_width = p_width,
                    plot_height = p_height,
                    x_range = x_range_pp,
                    y_range = y_range_pp,
                    min_border_left = min_border_large,
                    min_border_right = min_border_large,
                    min_border_bottom = min_border_large,
                    min_border_top = min_border_large,
                    toolbar_location = None,
                    title = '')

pp_left.xaxis.axis_label = x_label_pp
pp_left.yaxis.axis_label = y_label_pp
pp_left.grid.grid_line_color = None
pp_left.axis.minor_tick_in = False
pp_left.axis.minor_tick_out = False
pp_left.axis.major_tick_in = False
pp_left.outline_line_alpha = 0
pp_left.xaxis.ticker = x_axis_ticker
pp_left.yaxis.ticker = y_axis_ticker
pp_left.background_fill_color = bg_color
pp_left.axis.axis_label_standoff = 10
pp_left.axis.axis_label_text_font_style = 'normal'
dict_label = {}
txt_label = ['pi','pi/2','0','-pi/2','-pi']
for val_num, val in enumerate(y_axis_ticker):
    dict_label.update({val: txt_label[val_num]})
pp_left.yaxis.major_label_overrides = dict_label

# plot contra-lateral hemifield
pp_left.quad(bottom = -np.pi/2, left = 0, right = 1, top = np.pi/2, color = stim_color)

# plot values
pp_left.circle(x = scatter_roi_long_left_norm, y = scatter_angle_left, line_color = scatter_angle_norm_left_col,line_alpha = 0.5,fill_color = None,size = scatter_size_dots_left,line_width = 0.5)
pp_left.multi_line(bin_angle_left_ebx_norm,bin_angle_left_eb,line_color = 'black',line_width = 1)
pp_left.line(x = bin_roi_long_left_norm, y = bin_angle_left, color = 'black',line_width = 1)
pp_left.circle(x = bin_roi_long_left_norm, y = bin_angle_left, line_color = 'black',fill_color = bin_angle_norm_left_col,size = bin_size_dots_left,line_width = 1)

pp_right = figure(  plot_width = p_width,
                    plot_height = p_height,
                    x_range = x_range_pp,
                    y_range = y_range_pp,
                    min_border_left = min_border_large,
                    min_border_right = min_border_large,
                    min_border_bottom = min_border_large,
                    min_border_top = min_border_large,
                    toolbar_location = None,
                    title = '')

pp_right.xaxis.axis_label = x_label_pp
pp_right.yaxis.axis_label = y_label_pp
pp_right.grid.grid_line_color = None
pp_right.axis.minor_tick_in = False
pp_right.axis.minor_tick_out = False
pp_right.axis.major_tick_in = False
pp_right.outline_line_alpha = 0
pp_right.xaxis.ticker = x_axis_ticker
pp_right.yaxis.ticker = y_axis_ticker
pp_right.background_fill_color = bg_color
pp_right.axis.axis_label_standoff = 10
pp_right.axis.axis_label_text_font_style = 'normal'
dict_label = {}
txt_label = ['pi','pi/2','0','-pi/2','-pi']
for val_num, val in enumerate(y_axis_ticker):
    dict_label.update({val: txt_label[val_num]})
pp_right.yaxis.major_label_overrides = dict_label

# plot contra-lateral hemifield
pp_right.quad(bottom = -np.pi/2, left = 0, right = 1, top = np.pi/2, color = stim_color)

# plot values
pp_right.circle(x = scatter_roi_long_right_norm, y = scatter_angle_right, line_color = scatter_angle_norm_right_col,line_alpha = 0.5,fill_color = None,size = scatter_size_dots_right,line_width = 0.5)
pp_right.multi_line(bin_angle_right_ebx_norm,bin_angle_right_eb,line_color = 'black',line_width = 1)
pp_right.line(x = bin_roi_long_right_norm, y = bin_angle_right, color = 'black',line_width = 1)
pp_right.circle(x = bin_roi_long_right_norm, y = bin_angle_right, line_color = 'black',fill_color = bin_angle_norm_right_col,size = bin_size_dots_right,line_width = 1)

# f = column(row(zoom_left,pp_left),row(zoom_right,pp_right))
f = column(row(pp_left),row(pp_right))
output_file("polar_progress_ang.html", title="Polar progression ANG")
show(f)

In [8]:
# plot rotation in bokeh
# ----------------------
rot_left = figure(  plot_width = p_width,
                    plot_height = p_height,
                    x_range = (-200,200),
                    y_range = (-200,200),
                    min_border_left = min_border_large,
                    min_border_right = min_border_large,
                    min_border_bottom = min_border_large,
                    min_border_top = min_border_large,
                    toolbar_location = None,
                    title = '')

rot_left.xaxis.axis_label = ''
rot_left.yaxis.axis_label = ''
rot_left.grid.grid_line_color = None
rot_left.axis.minor_tick_in = False
rot_left.axis.minor_tick_out = False
rot_left.axis.major_tick_in = False
rot_left.outline_line_alpha = 0
rot_left.background_fill_color = bg_color
rot_left.axis.axis_label_standoff = 10
rot_left.axis.axis_label_text_font_style = 'normal'

# fig.circle(ang_coord_left[:,0],ang_coord_left[:,1],color = 'gray')
# fig.circle(ang_coord_left[:,0],ang_coord_left[:,1],color = ang_norm_col_left,alpha = alpha_left,line_color = None)
# fig.circle(-70,ang_coord_left[:,1],color = ang_norm_col_left,alpha = alpha_left,line_color = None)

rot_left.circle(ang_coord_rot_left[:,0],ang_coord_rot_left[:,1],color = 'gray')
rot_left.circle(ang_coord_rot_left[:,0],ang_coord_rot_left[:,1],color = ang_norm_col_left,alpha = alpha_left,line_color = None)
rot_left.circle(-70,ang_coord_rot_left[:,1],color = ang_norm_col_left,alpha = alpha_left,line_color = None)

rot_right = figure(  plot_width = p_width,
                    plot_height = p_height,
                    x_range = (-200,200),
                    y_range = (-200,200),
                    min_border_left = min_border_large,
                    min_border_right = min_border_large,
                    min_border_bottom = min_border_large,
                    min_border_top = min_border_large,
                    toolbar_location = None,
                    title = '')

rot_right.xaxis.axis_label = ''
rot_right.yaxis.axis_label = ''
rot_right.grid.grid_line_color = None
rot_right.axis.minor_tick_in = False
rot_right.axis.minor_tick_out = False
rot_right.axis.major_tick_in = False
rot_right.outline_line_alpha = 0
rot_right.background_fill_color = bg_color
rot_right.axis.axis_label_standoff = 10
rot_right.axis.axis_label_text_font_style = 'normal'

# fig = figure(   plot_width = p_width,
#                 plot_height = p_height,
#                 x_range = (50,150),
#                 y_range = (-50,50),
#                 toolbar_location = None)

# fig.circle(ang_coord_right[:,0],ang_coord_right[:,1],color = 'gray')
# fig.circle(ang_coord_right[:,0],ang_coord_right[:,1],color = ang_norm_col_right,alpha = alpha_right,line_color = None)
# fig.circle(80,ang_coord_right[:,1],color = ang_norm_col_right,alpha = alpha_right,line_color = None)

rot_right.circle(ang_coord_rot_right[:,0],ang_coord_rot_right[:,1],color = 'gray')
rot_right.circle(ang_coord_rot_right[:,0],ang_coord_rot_right[:,1],color = ang_norm_col_right,alpha = alpha_right,line_color = None)
rot_right.circle(77,ang_coord_rot_right[:,1],color = ang_norm_col_right,alpha = alpha_right,line_color = None)

show(column(row(rot_left,rot_right)))
